In [0]:
import numpy as np # linear algebra
from numpy import log

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns # Visualization
import matplotlib.pyplot as plt # Visualization

import math

from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pylab import rcParams

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller #for Stationarity


import pmdarima as pm # Auto TimeSeries

from sklearn.metrics import mean_absolute_error, mean_squared_error #Error Metrics
import warnings # Supress warnings 
warnings.filterwarnings('ignore')

In [0]:
import seaborn as sns

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")

#string to date format
df['Month'] = pd.to_datetime(df['Month'],infer_datetime_format=True)
df = df.set_index(['Month'])
df.head(5)

In [0]:
df.shape

In [0]:
df.rename(columns = {'#Passengers':'Passengers'}, inplace = True)

In [0]:
df.head(5)

In [0]:
# ### visualizing graph to see the components

In [0]:
plt.figure(figsize=(15,7))
plt.title("Number of Airline Passengers by Years")
plt.xlabel('Year')
plt.ylabel('Passengers')
plt.plot(df)
plt.show()

In [0]:
df_train = df[:134]
df_test = df[134:]

In [0]:
#### Stationarity check

In [0]:
def stationarity_test(timeseries):
    # Get rolling statistics for window = 12 i.e. yearly statistics
    rolling_mean = timeseries.rolling(window = 12).mean()
    rolling_std = timeseries.rolling(window = 12).std()
    
    # Plot rolling statistic
    plt.figure(figsize= (10,6))
    plt.xlabel('Years')
    plt.ylabel('Open')    
    plt.title('Stationary Test: Rolling Mean and Standard Deviation')
    plt.plot(timeseries, color= 'blue', label= 'Original')
    plt.plot(rolling_mean, color= 'green', label= 'Rolling Mean')
    plt.plot(rolling_std, color= 'red', label= 'Rolling Std')   
    plt.legend()
    plt.show()
    
    # Dickey-Fuller test
    print('Results of Dickey-Fuller Test')
    df_test = adfuller(timeseries)

    df_output = pd.Series(df_test[0:4], index = ['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in df_test[4].items():
        df_output['Critical Value (%s)' %key] = value
    print(df_output)

    adf = df_test[0]
    pvalue = df_test[1]
    critical_value = df_test[4]['5%']
    if(pvalue < 0.05) and (adf < critical_value):
        print("The Series is STATIONARY")
    else:
        print("The Series is NOT Stationary")

In [0]:
# Lets test the stationarity score with original series data
stationarity_test(df_train)

In [0]:
### Using Differencing to make data Stationary

In [0]:
df_train_diff = df_train.diff(periods = 1) # First order differencing
plt.xlabel('Years')
plt.ylabel('Passengers')    
plt.title('Convert Non Stationary Data to Stationary Data using Differencing ')
plt.plot(df_train_diff)

In [0]:
df_train_diff.dropna(inplace = True)# Data transformation may add na values
stationarity_test(df_train_diff)


In [0]:
#### Likewise we can 2nd order differencing(if the data is not stationary in the 1st order differencing

In [0]:
df_train_diff2 = df_train_diff.diff(periods = 1) # second order differencing
plt.xlabel('Years')
plt.ylabel('Passengers')    
plt.title('Convert Non Stationary Data to Stationary Data using 2nd Differencing ')
plt.plot(df_train_diff2)

In [0]:
df_train_diff2.dropna(inplace = True)# Data transformation may add na values
stationarity_test(df_train_diff2)

In [0]:
### Additionally we can use transformation techniques like Log, exponential etc for smoothing the data.

In [0]:

df_train_log = np.log(df_train)

plt.subplot(211)
plt.plot(df_train, label= 'Time Series with Variance')
plt.legend()
plt.subplot(212)
plt.plot(df_train_log, label='Time Series without Variance (Log Transformation)')
plt.legend()  
plt.show()


In [0]:
stationarity_test(df_train_log)


In [0]:
### Rolling mean on the log data (Moving Average)

In [0]:
df_train_log_moving_avg = df_train_log.rolling(window = 12).mean()
plt.xlabel('Years')
plt.ylabel('Open')    
plt.title('Convert Non Stationary Data to Stationary Data using Moving Average')
plt.plot(df_train_log, color= 'blue', label='Orignal')
plt.plot(df_train_log_moving_avg, color= 'red', label='Moving Average')
plt.legend()

In [0]:
df_train_log_moving_avg_diff = df_train_log - df_train_log_moving_avg
df_train_log_moving_avg_diff.dropna(inplace = True)
stationarity_test(df_train_log_moving_avg_diff)


In [0]:
fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
plot_acf(df_train_diff2['Passengers'].tolist(), lags=20, ax=axes[0])
plot_pacf(df_train_diff2['Passengers'].tolist(), lags=20, ax=axes[1])

In [0]:
# Take the first difference to remove to make the process stationary
#data_diff = df.Passengers - df.Passengers.shift(1)

In [0]:
#### Building ARIMA

In [0]:
from statsmodels.tsa.arima.model import ARIMA


In [0]:
model1 = ARIMA(df_train, order = (14,2,6)) 
results_ARIMA = model1.fit()
print(results_ARIMA.summary())


In [0]:
results_ARIMA.plot_diagnostics(figsize=(10,8))
plt.show()


In [0]:
pred = results_ARIMA.predict()

In [0]:
type(pred)

In [0]:
pred

In [0]:
pred = pred.to_frame()
pred.head()

In [0]:
pred.head(5)

In [0]:
pred.tail(5)

In [0]:
print("Train MAPE: ",mean_absolute_percentage_error(df_train.Passengers,pred.predicted_mean))

In [0]:
print("Train MSE: ",mean_squared_error(df_train.Passengers,pred.predicted_mean))

In [0]:
forecast_test = results_ARIMA.forecast(10)

In [0]:
forecast_test = forecast_test.to_frame()
forecast_test.head()

In [0]:
forecast_test.head(10)

In [0]:
print("Test MAPE: ",mean_absolute_percentage_error(df_test.Passengers,forecast_test.predicted_mean))

In [0]:
print("Test MSE: ",mean_squared_error(df_test.Passengers,forecast_test.predicted_mean))

In [0]:
plt.figure(figsize= (10,6))
plt.plot(df_train.Passengers)
plt.plot(df_test.Passengers,color="orange")
plt.plot(forecast_test.predicted_mean,color="darkgreen")
plt.xlabel('Years')
plt.ylabel('Open stocks')
plt.title('forecasting the Time Series')

In [0]:
#### Auto ARIMA on Train dataset

In [0]:
model2 = pm.auto_arima(df_train, 
                      trace=True,
                      )

print(model2.summary())

In [0]:
model2.plot_diagnostics()

In [0]:
test_pred_2 = model2.predict()

In [0]:
test_pred_2= test_pred_2.to_frame()
test_pred_2.head()

In [0]:
test_pred_2.rename(columns={0:"Prediction"},inplace=True)

In [0]:
print("Test MAPE on Model2: ", mean_absolute_percentage_error(df_test.Passengers,test_pred_2.Prediction))

In [0]:
print("Test MSE on Model2: ", mean_squared_error(df_test.Passengers,test_pred_2.Prediction))

In [0]:
##In Multiplicative model, we assume that all the components of the time series are dependent and in Additive model it is assumed that the components are independent.

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose


In [0]:
# Multiplicative Decomposition 

In [0]:
multiplicative_decomposition = seasonal_decompose(df['Passengers'], model='multiplicative', period=30)

In [0]:
# Additive Decomposition
additive_decomposition = seasonal_decompose(df['Passengers'], model='additive', period=30)

# Plot
plt.rcParams.update({'figure.figsize': (16,12)})
multiplicative_decomposition.plot().suptitle('Multiplicative Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

additive_decomposition.plot().suptitle('Additive Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

In [0]:
### Detrend

In [0]:
# Using scipy: Subtract the line of best fit using least squares
from scipy import signal
detrended = signal.detrend(df['Passengers'].values)
plt.plot(detrended)
plt.title('Air Passengers detrended ', fontsize=16)

In [0]:
# Using statmodels: Subtracting the Trend Component
from statsmodels.tsa.seasonal import seasonal_decompose
result_mul = seasonal_decompose(df['Passengers'], model='multiplicative', period=30)
detrended = df['Passengers'].values / result_mul.trend
plt.plot(detrended)
plt.title('Air Passengers detrended by subtracting the trend component', fontsize=16)

In [0]:
# Subtracting the seasonal Component

# Time Series Decomposition
result_mul = seasonal_decompose(df['Passengers'], model='multiplicative', period=30)


# Deseasonalize
deseasonalized = df['Passengers'].values / result_mul.seasonal

# Plot
plt.plot(deseasonalized)
plt.title('Air Passengers Deseasonalized', fontsize=16)
plt.plot()

In [0]:
# Substracting Seasoanl component
# Time Series Decomposition
result_add = seasonal_decompose(df['Passengers'], model='additive', period=30)

# Deseasonalize
deseasonalized_add = df['Passengers'].values - result_add.seasonal

# Plot
plt.plot(deseasonalized_add)
plt.title('Air Passengers Deseasonalized', fontsize=16)
plt.plot()


In [0]:
desea_df = deseasonalized.to_frame()



In [0]:
desea_df

In [0]:
train_desea = desea_df[:134]
test_desea = desea_df[134:]

In [0]:
#### Testing Stationarity on data after removing decomposition value


In [0]:
stationarity_test(deseasonalized)

In [0]:
deseasonalized_diff = deseasonalized.diff(periods = 1) # First order differencing
plt.xlabel('Years')
plt.ylabel('Passengers')    
plt.title('Convert Non Stationary Data to Stationary Data using Differencing on deseasoned data')
plt.plot(deseasonalized_diff)

In [0]:
deseasonalized_diff.dropna(inplace = True)# Data transformation may add na values

stationarity_test(deseasonalized_diff)

In [0]:
deseasonalized_diff2 = deseasonalized_diff.diff(periods = 1) # Second order differencing
plt.xlabel('Years')
plt.ylabel('Passengers')    
plt.title('Convert Non Stationary Data to Stationary Data using 2nd order Differencing on deseasoned data')
plt.plot(deseasonalized_diff2)

In [0]:
deseasonalized_diff2.dropna(inplace = True)# Data transformation may add na values

stationarity_test(deseasonalized_diff2)


In [0]:
deseasonalized_diff2

In [0]:
deseasonalized_diff2 = deseasonalized_diff2.to_frame()
deseasonalized_diff2.head()


In [0]:
#### Auto ARIMA On Decomposed Multiplicative data


In [0]:
model_3 = pm.auto_arima(train_desea,trace=True)
print(model_3.summary())


In [0]:
model_3.plot_diagnostics()


In [0]:
pred_test_sea_3 = model_3.predict()

In [0]:
pred_test_sea_3 = pred_test_sea_3.to_frame()
pred_test_sea_3.head()

In [0]:
pred_test_sea_3.rename(columns={0:"Prediction"},inplace=True)

In [0]:
print("Test MAPE : ",mean_absolute_percentage_error(test_desea.seasonal,pred_test_sea_3.Prediction)) #without Seasonality component to predictions

In [0]:
print("Train MSE : ",mean_squared_error(test_desea.seasonal,pred_test_sea_3.Prediction)) #without Seasonality component to predictions

In [0]:
 #### Additive Train test split


In [0]:
desea_df_add = deseasonalized_add.to_frame()
desea_df_add.head()


In [0]:
train_desea_add = desea_df_add[:134]
test_desea_add = desea_df_add[134:]

In [0]:
stationarity_test(train_desea_add)

In [0]:
train_desea_add_diff = train_desea_add.diff(periods = 1) # First order differencing
plt.xlabel('Years')
plt.ylabel('Passengers')    
plt.title('Convert Non Stationary Data to Stationary Data using Differencing on deseasoned data (additive)')
plt.plot(train_desea_add_diff)


In [0]:
train_desea_add_diff.dropna(inplace = True)# Data transformation may add na values

stationarity_test(train_desea_add_diff)

In [0]:
### Auto ARIMA on Decomposed Additive data


In [0]:
model_4 = pm.auto_arima(train_desea_add,trace=True)

print(model_4.summary())


In [0]:
model_4.plot_diagnostics()

In [0]:
pred_test_sea_4 = model_4.predict()

In [0]:
pred_test_sea_4 = pred_test_sea_4.to_frame()
pred_test_sea_4.head()


In [0]:
pred_test_sea_4.rename(columns={0:"Prediction"},inplace=True)

In [0]:
print("Test MAPE : ",mean_absolute_percentage_error(test_desea_add.seasonal,pred_test_sea_4.Prediction)) #without Seasonality component to predictions

In [0]:
print("Test MSE : ",mean_squared_error(test_desea_add.seasonal,pred_test_sea_4.Prediction)) #without Seasonality component to predictions


In [0]:
#### SARIMAX

In [0]:
model_5 = sm.tsa.statespace.SARIMAX(df_train,order=(2,1,2), seasonal_order=(2,1,2,12),

                                 enforce_stationarity=False, enforce_invertibility=False,).fit()

model_5.summary()

In [0]:
#model_5 = pm.auto_arima(df_train,seasonality=True,trace=True)

#print(model_5.summary())


In [0]:
model_5.plot_diagnostics()


In [0]:
pred_train_sea_5 = model_5.predict()

In [0]:
pred_train_sea_5 = pred_train_sea_5.to_frame()
pred_train_sea_5.head()


In [0]:
#pred_train_sea_5.rename(columns={0:"Prediction"},inplace=True)


In [0]:
print("Test MAPE : ",mean_absolute_percentage_error(df_train.Passengers,pred_train_sea_5.predicted_mean)) #without Seasonality component to predictions


In [0]:
print("Test MSE : ",mean_squared_error(df_train.Passengers,pred_train_sea_5.predicted_mean)) #without Seasonality component to predictionshow t

In [0]:
forecast_test = model_5.forecast(10)

In [0]:
forecast_test = forecast_test.to_frame()

In [0]:
print("Test MAPE : ",mean_absolute_percentage_error(df_test.Passengers,forecast_test.predicted_mean)) #without Seasonality component to predictions

In [0]:
print("Test MSE : ",mean_squared_error(df_test.Passengers,forecast_test.predicted_mean)) #without Seasonality component to predictionshow t

In [0]:
pred_train = model_5.predict(df_train.index.min(), df_train.index.max())

pred_test = model_5.predict(df_test.index.min(), df_test.index.max())

#plt.title('ARIMA model MSE:{}'.format(mean_squared_error(test,pred_test)))

plt.figure(figsize=(16,8))

plt.plot(df_train, label='train')

plt.plot(pred_train, color='orange', linestyle='--', label= 'train prediction')

plt.plot(pred_test, color='red', linestyle='--', label= 'prediction')

plt.plot(df_test, color='green', label='actual')

plt.plot(forecast_test.predicted_mean, color='purple', linestyle='-', label= 'forecast')

plt.legend(loc='best')

plt.show()